In [1]:
import json
import pandas as pd
import numpy as np
import time
import requests
from collections import defaultdict
import logging

In [2]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("economic-test")
logger.setLevel(logging.INFO)

In [3]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [4]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [5]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [6]:
def getCurrentAndLastBlock():
    block = getBlockNumber()
    last_block = getLastBlockOfCurrentEpoch()
    return block, last_block

In [7]:
def getEpoch():
    method = 'hmy_getEpoch'
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [8]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [0]
    return get_information(method, params)['result'] 

In [9]:
def getStakeAndUndelegate2(epoch):
    validator = dict()
    undelegate = dict()
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        address = i['validator']['address']
        stake = dict()
        undel = dict()
        for d in i['validator']['delegations']:
            del_address = d['delegator-address']
            del_amount = d['amount']
            if not d['undelegations']:
                undel_amount = 0
            flag = False
            for j in d['undelegations']:
                if epoch == j['epoch']:
                    flag = True
                    undel_amount = j['amount']
                    break
            if not flag:
                undel_amount = 0
            
            stake[del_address] = del_amount
            undel[del_address] = undel_amount
        validator[address] = stake
        undelegate[address] = undel
    return validator, undelegate

In [10]:
def diffAndFilter2(map1, map2):
    map3 = dict()
    for key, val in map2.items():
        diff = dict()
        for k, v in map2[key].items():
            if key in map1 and k in map1[key]:
                diff[k] = v - map1[key][k]
            if key in map1 and k not in map1[key]:
                diff[k] = map2[key][k]
        map3[key] = diff
    return map3

In [11]:
def U1_test(single):
    logger.info("Test-U1: Delegator/validator stake locked until undelegate")
    num = 1
    
    block, last_block = getCurrentAndLastBlock()
    logger.info(f"current and last block numbers: {block}, {last_block}")
    if block + num > last_block:  
        logger.info(f"wait until new epoch starts ...")
        new_block = last_block + 1
        while block < new_block:
            block = getBlockNumber()
    iterations = 0
    flag = True
    total_reduce_num = 0
    while iterations < num:
        epoch = getEpoch()
        logger.info(f"current epoch number, {epoch}, current block number, {block}, will begin testing...")
        stake, undelegate = getStakeAndUndelegate2(epoch)
        next_block = block + 1
        while block < next_block:
            block = getBlockNumber()
        epoch = getEpoch()
        logger.info(f"next block reached, {block}, current epoch, {epoch}, will compare the stakes")
        new_stake, new_undelegate = getStakeAndUndelegate2(epoch)
        diff_stake = diffAndFilter2(stake, new_stake)
        diff_undelegate = diffAndFilter2(undelegate, new_undelegate)

        reduce_num = 0
        for key, val in diff_stake.items():
            for k,v in diff_stake[key].items():
                if v < 0:
                    reduce_num += 1
                    total_reduce_num +=1
                    if diff_undelegate[key][k] <= 0:
                        logger.warning(f"Test-U1: Fail")
                        logger.warning(f"Delgeator stake reduces without undelegate")
                        logger.warning(f"undelegate changes:  {diff_undelegate[key][k]}")
                        logger.warning(f"stake changes: {v}\n")
                        flag = False        
        if reduce_num == 0:
            logger.info(f"No stake reduces at current test, need more tests\n")
        iterations += 1  
    if single:
        curr_test = None
    else:        
        curr_test = U2_test
    if total_reduce_num == 0:
        return "Need More Tests", curr_test
    if flag:
        logger.info(f"Test-U1: Succeed\n")
        return True, curr_test
    if not flag:
        return False, curr_test

In [12]:
U1_test(True)

INFO:economic-test:Test-U1: Delegator/validator stake locked until undelegate
INFO:economic-test:current and last block numbers: 934, 1349
INFO:economic-test:current epoch number, 2, current block number, 934, will begin testing...
INFO:economic-test:next block reached, 936, current epoch, 2, will compare the stakes
INFO:economic-test:No stake reduces at current test, need more tests



('Need More Tests', None)